In [1]:
# 1. Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 2. Importando bibliotecas necessárias
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam


In [3]:
# 3. Definir o diretório do dataset (altere o caminho se necessário)
dataset_dir = "/content/drive/MyDrive/bancodedados_seg_linha/"

In [4]:
!ls "/content/drive/MyDrive/bancodedados_seg_linha/"

forward  left  nothing	right


In [5]:
# 4. Definir parâmetros de treinamento
EPOCHS = 15
INIT_LR = 1e-3
BS = 32

In [6]:
# 5. Criar ImageDataGenerator com divisão entre treino e validação
datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(28, 28),
    color_mode="grayscale",  # Imagens em escala de cinza
    batch_size=BS,
    class_mode="categorical",  # 4 classes (forward, left, right, nothing)
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(28, 28),
    color_mode="grayscale",
    batch_size=BS,
    class_mode="categorical",
    subset='validation'
)

Found 223 images belonging to 4 classes.
Found 53 images belonging to 4 classes.


In [7]:
# 6. Definir a arquitetura do modelo (LeNet-5 modificado)
def build_lenet(width, height, depth, classes):
    model = Sequential()
    inputShape = (height, width, depth)

    # Primeira camada: Convolução -> ReLU -> Max Pooling
    model.add(Conv2D(20, (5, 5), padding="same", use_bias=False, input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Segunda camada: Convolução -> ReLU -> Max Pooling
    model.add(Conv2D(50, (5, 5), padding="same", use_bias=False))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Camada totalmente conectada
    model.add(Flatten())
    model.add(Dense(500))
    model.add(Activation("relu"))

    # Camada de saída com ativação softmax para 4 classes
    model.add(Dense(classes))
    model.add(Activation("softmax"))

    return model

In [8]:
# Construir e compilar o modelo
model = build_lenet(28, 28, 1, 4)

# Corrigindo a sintaxe do Adam
opt = Adam(learning_rate=INIT_LR)

# Compilar o modelo
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

print("[INFO] Modelo compilado!")

# 7. Treinar o modelo
print("[INFO] Iniciando o treinamento...")
H = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    verbose=1
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


[INFO] Modelo compilado!
[INFO] Iniciando o treinamento...
Epoch 1/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 110s 16s/step - accuracy: 0.3860 - loss: 1.1998 - val_accuracy: 0.7170 - val_loss: 0.8757
Epoch 2/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 31s 4s/step - accuracy: 0.8565 - loss: 0.5793 - val_accuracy: 0.7547 - val_loss: 0.5436
Epoch 3/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 47s 5s/step - accuracy: 0.9428 - loss: 0.1554 - val_accuracy: 0.9245 - val_loss: 0.2331
Epoch 4/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 31s 5s/step - accuracy: 1.0000 - loss: 0.0351 - val_accuracy: 1.0000 - val_loss: 0.0566
Epoch 5/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 32s 5s/step - accuracy: 1.0000 - loss: 0.0027 - val_accuracy: 1.0000 - val_loss: 0.0219
Epoch 6/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 28s 4s/step - accuracy: 1.0000 - loss: 4.7033e-04 - val_accuracy: 1.0000 - val_loss: 0.0054
Epoch 7/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 34s 5s/step - accuracy: 1.0000 - loss: 1.2364e-04 - val_accuracy: 1.0000 - val_loss: 0.0025
Epoch 8/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 36s 5s/step - accuracy: 1.0

In [9]:
# Avaliar o modelo no conjunto de validação
loss, accuracy = model.evaluate(validation_generator)

# Exibir a acurácia em porcentagem
print(f"Acurácia do modelo: {accuracy * 100:.2f}%")


2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 1.0000 - loss: 0.0028
Acurácia do modelo: 100.00%


In [10]:
# 8. Salvar o modelo treinado
model.save("model_pi_2.h5")
print("[INFO] Modelo salvo como model_pi_2.h5")

[INFO] Modelo salvo como model_pi_2.h5


In [11]:
from google.colab import files
files.download("model_pi_2.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>